In [1]:
from __future__ import print_function
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles

In [2]:
import pandas as pd

In [3]:
outDir ='simple_tutorial'
dbFile = 'baseline2018a.db'
opsimdb = db.OpsimDatabase(dbFile)
resultsDb = db.ResultsDb(outDir=outDir)

In [4]:
# The pass metric just passes data straight through.
metric = metrics.PassMetric(cols=['filter', 'fieldRA','fieldDec', 'fiveSigmaDepth', 'observationStartMJD','night','airmass'])
# use a unit slicer to grab all of the data
slicer = slicers.UniSlicer()
# only use the first 7 nights
sql = 'night <= 7'
bundle = metricBundles.MetricBundle(metric, slicer, sql)

In [5]:
bg = metricBundles.MetricBundleGroup({0: bundle}, opsimdb, outDir=outDir, resultsDb=resultsDb)
bg.runAll()

Querying database SummaryAllProps with constraint night <= 7 for columns ['observationStartMJD', 'night', 'airmass', 'fieldRA', 'filter', 'fiveSigmaDepth', 'fieldDec']
Found 5196 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


/home/docmaf/lsst_repos/sims_maf/python/lsst/sims/maf/db/resultsDb.py:290: UserWarning: Warning! Cannot save non-conforming summary statistic.
  warnings.warn('Warning! Cannot save non-conforming summary statistic.')


In [6]:
#Put the metric values, in this case just the first 7 nights, into a pandas dataframe.

In [7]:
data_df = pd.DataFrame(bundle.metricValues.data[0])

In [8]:
data_df.head()

,observationStartMJD,night,airmass,fieldRA,filter,fiveSigmaDepth,fieldDec
0,59853.016794,1,1.004789,305.088793,z,23.348592,-24.889283
1,59853.017801,1,2.017072,302.892080,z,22.545454,29.966322
2,59853.019826,1,1.186596,305.482200,z,23.167857,-62.802603
3,59853.020278,1,1.152827,307.103419,z,23.201599,-60.072224
4,59853.020729,1,1.123414,308.429608,z,23.231315,-57.316612


In [9]:
small_sample = []

In [10]:
# Loop trough and only get the first 2 observations on each night

In [11]:
for n in np.unique(data_df.night.values):
    small_sample.append(data_df[data_df.night == n][0:2])

In [12]:
mydb = pd.concat(small_sample)

In [13]:
#change the col order to be a little nicer

In [14]:
mydb[['night','observationStartMJD','filter','fieldRA','fieldDec','fiveSigmaDepth','airmass']]

,night,observationStartMJD,filter,fieldRA,fieldDec,fiveSigmaDepth,airmass
0,1,59853.016794,z,305.088793,-24.889283,23.348592,1.004789
1,1,59853.017801,z,302.892080,29.966322,22.545454,2.017072
772,2,59853.985324,z,294.547736,12.697577,22.647813,1.366074
773,2,59853.985764,z,292.965911,14.992815,22.599638,1.421099
1613,3,59854.984688,y,294.547736,12.697577,21.585365,1.366489
1614,3,59854.985139,y,292.965911,14.992815,21.529766,1.422345
1874,4,59855.986262,z,296.918411,-58.380321,22.623693,1.134010
1875,4,59855.986713,z,294.488727,-61.010990,22.625517,1.164413
2707,5,59856.985532,z,289.582803,0.187415,22.329336,1.172105
2708,5,59856.985984,z,287.999985,2.635102,22.300933,1.209209


# Match MaxMetirc('airmass') example in slides
This will match the dataSlice 1 table in the slides

In [15]:
# The pass metric just passes data straight through.
metric1 = metrics.MaxMetric(col='airmass')
# use a unit slicer to grab all of the data
slicer1 = slicers.OneDSlicer(sliceColName='night',binsize=1)
# Use stric slq contraint so we only get the very first 2 observations in the table.
sql1 = 'night <= 7 and filter = "z" and observationStartMJD <= 59853.017802'
bundle1 = metricBundles.MetricBundle(metric1, slicer1, sql1)

In [16]:
bg1 = metricBundles.MetricBundleGroup({0: bundle1}, opsimdb, outDir=outDir, resultsDb=resultsDb)
bg1.runAll()

Querying database SummaryAllProps with constraint night <= 7 and filter = "z" and observationStartMJD <= 59853.017802 for columns ['night', 'airmass']
Found 2 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


/home/docmaf/lsst_repos/sims_maf/python/lsst/sims/maf/slicers/oneDSlicer.py:61: UserWarning: binMin = binMax (maybe your data is single-valued?). Increasing binMax by 1 (or 2*binsize, if binsize set).
  warnings.warn('binMin = binMax (maybe your data is single-valued?). '


Note from the output above that only 2 visits were found `Found 2 visits`.

Grab the metric data, and see that it is indeed the maximum airmass we expect and it matches what is shown on the slides.

In [17]:
bundle1.metricValues.compressed()

array([ 2.01707179])

# Match MaxMetirc('fiveSigmaDepth') example in slides
This will match the dataSlice 1 table in the slides

In [32]:
# The pass metric just passes data straight through.
metric2 = metrics.MaxMetric(col='fiveSigmaDepth')
# use a unit slicer to grab all of the data
slicer2 = slicers.OneDSlicer(sliceColName='night',binsize=1)
# Use stric slq contraint so we only get the very first 2 observations in the table.
sql2 = 'night <= 7 and filter = "z" and observationStartMJD <= 59853.017802'
bundle2 = metricBundles.MetricBundle(metric2, slicer2, sql2)

In [28]:
bg2 = metricBundles.MetricBundleGroup({0: bundle2}, opsimdb, outDir=outDir, resultsDb=resultsDb)
bg2.runAll()

Querying database SummaryAllProps with constraint night <= 7 and filter = "z" and observationStartMJD <= 59853.017802 for columns ['fiveSigmaDepth', 'night']
Found 2 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


/home/docmaf/lsst_repos/sims_maf/python/lsst/sims/maf/slicers/oneDSlicer.py:61: UserWarning: binMin = binMax (maybe your data is single-valued?). Increasing binMax by 1 (or 2*binsize, if binsize set).
  warnings.warn('binMin = binMax (maybe your data is single-valued?). '


In [30]:
bundle2.metricValues.compressed()

array([ 23.34859188])